- This code has written by Ahmadreza Attarpour (a.attarpour@mail.utoronto.ca)
- It's a practice to check how we can use LangChain/LangGraph to build tools and agents

-------------------------------------

# ReAct Agent

- Loop of observation, action and thought
- The agent uses a combination reasoning steps to guide actions in real-time, using feedback from those actions to further inform the next step in reasoning


Prompt --> reasoning --> actions --> observation and reasoning --> repeat



In [1]:
import os

In [2]:
# load the environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Read the variable from the environment and set it as a env variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
if LANGCHAIN_PROJECT:
    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
if os.getenv("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
if os.getenv("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
if os.getenv("SERPER_API_KEY"):
    os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [4]:
# load the LLM and embeddings from langchain_google_genai
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
# This is a sentence-transformers model: It maps sentences & paragraphs to a
# 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain_groq import ChatGroq
llm = ChatGroq(model="Gemma2-9b-It")
# llm = ChatGroq(model="llama3-70b-8192")

/Users/ahmadreza/anaconda3/envs/aa_agent/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
llm.invoke("What is the capital of Iran?")

AIMessage(content='The capital of Iran is **Tehran**. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 16, 'total_tokens': 29, 'completion_time': 0.023636364, 'prompt_time': 0.001245901, 'queue_time': 0.102435938, 'total_time': 0.024882265}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c3f26dac-8d15-4b76-80ce-2953618e4631-0', usage_metadata={'input_tokens': 16, 'output_tokens': 13, 'total_tokens': 29})

In [6]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor, tool
from langchain import hub



In [12]:
# create the search tool
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="GoogleSearch",
        func=search.run,
        description="Useful for when you need to answer questions with search.",
        verbose=True
    )
]

In [13]:
# create the prompt template
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Observation Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought: {agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [14]:
# build the agent
agent = create_react_agent(llm, tools, prompt)

In [15]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

In [24]:
agent_executor.invoke({
    "input": "I have huge light sheet data of cFos, what's the best way to segment and analyze it?"
})



> Entering new AgentExecutor chain...
Thought: This question requires knowledge about image analysis techniques specifically for light sheet microscopy data of cFos expression. I need to research common segmentation and analysis methods used for this type of data.

Action: GoogleSearch
Action Input: "light sheet microscopy cFos segmentation analysis"Advances in microscopy, such as light-sheet fluorescence microscopy ... segmentation maps are passed to the cluster-wise analysis module ... This work presents a valuable self-supervised method for the segmentation of 3D cells in microscopy images, alongside an implementation as a Napari plugin ... Light-sheet microscopy (LSM) has proven a useful tool in neuroscience to image whole brains with high frame rates at cellular resolution and, in combination ... We generated high-quality annotations of light-sheet microscopy data of cleared whole mouse brains stained for c-Fos in a VR environment. Next, ... We reasoned that volume imaging using

{'input': "I have huge light sheet data of cFos, what's the best way to segment and analyze it?",
 'output': "A good starting point for segmenting and analyzing your cFos light sheet data would be to explore the NEATmap software. It's a deep learning-based pipeline designed for this specific purpose and offers high-precision segmentation of cFos neurons in whole-brain imaging.",
 'intermediate_steps': [(AgentAction(tool='GoogleSearch', tool_input='light sheet microscopy cFos segmentation analysis', log='Thought: This question requires knowledge about image analysis techniques specifically for light sheet microscopy data of cFos expression. I need to research common segmentation and analysis methods used for this type of data.\n\nAction: GoogleSearch\nAction Input: "light sheet microscopy cFos segmentation analysis"'),
   'Advances in microscopy, such as light-sheet fluorescence microscopy ... segmentation maps are passed to the cluster-wise analysis module ... This work presents a valu

# custom tool

In [7]:
# custom tool
@tool
def get_employee_id(name: str) -> str:
    """Get the employee ID for a given name."""

    fake_employees = {
        "Alice": "12345",
        "Bob": "67890",
        "Charlie": "54321",
        "David": "98765",
        "Eve": "11223",
        "Frank": "44556",
        "Grace": "77889",
        "Heidi": "33445",
        "Ivan": "66778",
        "Judy": "88990"
    }
    # This is a mock implementation, replace with actual logic
    return fake_employees.get(name, "Employee not found")

@tool
def get_employee_salary(name: str) -> str:
    """Get the employee salary for a given name."""

    fake_salaries = {
        "Alice": "$100,000",
        "Bob": "$120,000",
        "Charlie": "$110,000",
        "David": "$130,000",
        "Eve": "$140,000",
        "Frank": "$150,000",
        "Grace": "$160,000",
        "Heidi": "$170,000",
        "Ivan": "$180,000",
        "Judy": "$190,000"
    }
    # This is a mock implementation, replace with actual logic
    return fake_salaries.get(name, "Salary not found")

In [8]:
# create the tools
tools = [
    get_employee_id,
    get_employee_salary
]

In [9]:
# get the prompt
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [10]:
# create the agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

In [13]:
# invoke the agent
agent_executor.invoke({
    "input": "what's the salary of Alice?"
})



> Entering new AgentExecutor chain...
Thought: I need to find Alice's employee ID first and then use it to get her salary.
Action: get_employee_id
Action Input: Alice12345Thought: Now that I have Alice's employee ID, I can get her salary.
Action: get_employee_salary
Action Input: Alice$100,000Thought: I now know the final answer
Final Answer: Alice's salary is $100,000 


> Finished chain.


{'input': "what's the salary of Alice?",
 'output': "Alice's salary is $100,000",
 'intermediate_steps': [(AgentAction(tool='get_employee_id', tool_input='Alice', log="Thought: I need to find Alice's employee ID first and then use it to get her salary.\nAction: get_employee_id\nAction Input: Alice"),
   '12345'),
  (AgentAction(tool='get_employee_salary', tool_input='Alice', log="Thought: Now that I have Alice's employee ID, I can get her salary.\nAction: get_employee_salary\nAction Input: Alice"),
   '$100,000')]}